


## As the SAM masks are not stored in the cluster, they have to be generated.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
from ultralytics import SAM

In [ ]:
folder_path = "/data/"
eval_path = folder_path + "cyprus_eval_frames.csv"
eval_path_results = folder_path + "/fixation_and_labels_total.csv"

In [ ]:
# sam model
model = SAM("/models/sam_b.pt") # path to the model
model.info()
sam = model

### create sam masks for all images


In [ ]:
data = pd.read_csv(eval_path_results)

for color in ["red"]:

    for i in range(0, len(data)):
        x = data["x"].iloc[i]
        y = data["y"].iloc[i]
        session = data["session"].iloc[i]
        response = data["response"].iloc[i]
        frame_nr = data["frame_nr"].iloc[i]
        str_frame_nr = f"{frame_nr:05d}"

        # get image path
        img_path = (
            folder_path
            + "/evalset/images_wo_dot/"
            + session
            + "_"
            + str_frame_nr
            + ".jpg"
        )

        result = sam(source=img_path, stream=False, points=[x, y], labels=[1])

        # save the masks
        # Convert to numpy array (Eager Execution)
        mask_array = result[0][0].masks.data.numpy()

        # Save to .npy file
        np.save(
            folder_path + "/data/evalset/sams/" + session + "_" + str_frame_nr + ".npy",
            mask_array,
        )

### Plot all images with sam Files

In [ ]:
# plot the image with mask for each image
data = pd.read_csv(eval_path_results)
for i in range(0, 100):
    x = data["x"].iloc[i]
    y = data["y"].iloc[i]
    session = data["session"].iloc[i]
    response = data["response"].iloc[i]
    frame_nr = data["frame_nr"].iloc[i]
    str_frame_nr = f"{frame_nr:05d}"
    print(session, str_frame_nr, x, y)

    img_path = (
        folder_path + "/evalset/images_wo_dot/" + session + "_" + str_frame_nr + ".jpg"
    )
    # Load the mask
    mask = np.load(
        folder_path + "/evalset/sams/" + session + "_" + str_frame_nr + ".npy"
    )

    # plot the image with mask

    plt.figure(figsize=(5, 15))
    plt.imshow(image)

    image = Image.open(img_path)
    img = np.array(image)
    img[mask[0] == False] = (255, 0, 0)  # Red color
    plt.imshow(img)
    plt.axis("off")  # Turn off x and y axes
    plt.show()